In [1]:
from Tools import *
from photoObject import *
import emcee
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import xmatch
from Tools import *
from sTools import *
import random
import os
import glob

sns.set_theme(style="white",font_scale = 1.5)


In [2]:
random.seed(420)

Fresco=fits.open("../Catalogs/fresco_only_n_v2.fits")
Zdic={}
for gal in Fresco[1].data:
    Zdic["N-"+str(gal["id"])]=[gal["z_spec"],gal["z_map"],[gal["ra"],gal["dec"]],gal["max_line_sn"]]
randomGals=random.choices(list(Zdic.keys()),k=1000)
vals=[Zdic[gal] for gal in randomGals]
Zdic= dict(zip(randomGals, vals))

In [3]:
z_fresco,z_spec,dz,coord,sn,id=[],[],[],[],[],[]
for k in Zdic.keys():
    #print(k,Zdic[k][0])
    z_spec.append(Zdic[k][0])
    z_fresco.append(Zdic[k][1])
    dz.append(Zdic[k][0]-Zdic[k][1])
    coord.append(Zdic[k][2])
    sn.append(Zdic[k][3])
    id.append(k)
    
#for i in id:
    #i=f"{int(i[2:]):05}"
    #os.system('wget -P ../data/large_files/FrescoDatabase/GOODS-N/ https://s3.amazonaws.com/grizli-v2/HST/Pipeline/fresco-only-n-v2/Extractions/fresco-only-n-v2_'+i+'.1D.fits')




In [4]:
id

['N-10772',
 'N-7597',
 'N-16880',
 'N-4661',
 'N-21934',
 'N-21780',
 'N-20754',
 'N-16666',
 'N-9916',
 'N-23401',
 'N-25006',
 'N-31865',
 'N-16766',
 'N-27877',
 'N-31912',
 'N-18539',
 'N-29896',
 'N-14501',
 'N-19431',
 'N-30004',
 'N-14879',
 'N-10128',
 'N-30186',
 'N-12995',
 'N-26150',
 'N-11453',
 'N-19718',
 'N-19149',
 'N-23227',
 'N-12779',
 'N-17576',
 'N-21067',
 'N-19002',
 'N-25279',
 'N-3725',
 'N-4202',
 'N-18766',
 'N-5101',
 'N-28017',
 'N-11826',
 'N-23133',
 'N-10907',
 'N-22086',
 'N-30997',
 'N-21902',
 'N-23464',
 'N-19727',
 'N-31241',
 'N-1310',
 'N-378',
 'N-7342',
 'N-18292',
 'N-9871',
 'N-11408',
 'N-18496',
 'N-11403',
 'N-10027',
 'N-9561',
 'N-6591',
 'N-28433',
 'N-14429',
 'N-13243',
 'N-25486',
 'N-14826',
 'N-20175',
 'N-21015',
 'N-27697',
 'N-27267',
 'N-13165',
 'N-20094',
 'N-2489',
 'N-23533',
 'N-1700',
 'N-14533',
 'N-21118',
 'N-26481',
 'N-21540',
 'N-3417',
 'N-28924',
 'N-15119',
 'N-19314',
 'N-17636',
 'N-6439',
 'N-30790',
 'N-13653

In [13]:
#id, coord, err between 40k and 50k



Data=[]
for i,k in zip(id,coord):
    fs=fits.open("../data/large_files/FrescoDatabase/GOODS-N/fresco-only-n-v2_"+f"{int(i[2:]):05}"+".1D.fits")
    wave=fs[1].data["wave"]
    mask=(wave>42000)&(wave<46000)
    err=np.median((fs[1].data["err"]/fs[1].data["flat"])[mask])
    Data.append([i,k,err])

np.save("../Catalogs/SpatialErr_Coord.npy",Data)




<ipython-input-13-0ae0a30e590a>:10: RuntimeWarning: invalid value encountered in true_divide
  err=np.median((fs[1].data["err"]/fs[1].data["flat"])[mask])
/Users/gonzalo/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:

binsize=1000
bin_edges = np.arange(min(Merged_data), max(Merged_data)+binsize, binsize)
bin_centers = bin_edges_to_centers(bin_edges)
binned_Data=bin_data(Merged_data,Merged_dz,bin_edges)
binned_voff=bin_data(Merged_data,Merged_voff,bin_edges)

MedianData=[]
for d in binned_Data:
    MedianData.append( np.percentile(d,q=[16,50,84]) )
MedianData=np.array(MedianData)

MedianVoff=[]
for d in binned_voff:
    MedianVoff.append( np.percentile(d,q=[16,50,84]) )
MedianVoff=np.array(MedianVoff)




In [ ]:
%load_ext autoreload
%autoreload

difference=-Tools.kmsToangstrom2(np.array(Merged_voff),np.array(Merged_data))

In [ ]:
plt.scatter(difference,Merged_voff)
plt.ylim(-500,500)
plt.xlim(-100,100)
plt.axhline(y=0,ls="--",label="0kms")
plt.axvline(x=0,ls="--")

plt.axhline(y=51,ls="--",color="red")
plt.axvline(x=7,ls="--",color="red",label="Median of velocity offsets")
plt.axvspan(-10,10, alpha=0.5, color='red',label="Dispersion 10A")
#plt.axhline(y=30,ls="--")
plt.xlabel("Wavelength Diffeerence A")
plt.ylabel("Velocity difference kms")
plt.legend()
plt.show()

In [ ]:
plt.hist(difference,bins=100,range=[-100,100])
plt.xlabel("Difference Wavelength")

In [ ]:
plt.scatter(Merged_sn,Merged_dz)
plt.axvline(x=8)
plt.axvline(x=20)
plt.xlabel("SN")
plt.ylabel("dz")
plt.show()

In [ ]:

plt.scatter(Merged_data,Merged_dz,alpha=0.04)
plt.errorbar(bin_centers,MedianData[:,1],yerr=[MedianData[:,1]-MedianData[:,0],MedianData[:,2]-MedianData[:,1]],marker="o",ls="--",color="#702963",label="Median values and p16,p84",ms=10)
plt.axhline(y=0,ls="--",color="gray")
plt.ylabel(r"$\Delta$ z (z$_{spec}$-z$_{fresco}$)")
plt.xlabel(r"z$_{spec}$")
plt.ylim(-0.05,0.05)
plt.legend()
plt.show()


plt.scatter(Merged_data,Merged_voff,alpha=0.04)
plt.errorbar(bin_centers,MedianVoff[:,1],yerr=[MedianVoff[:,1]-MedianVoff[:,0],MedianVoff[:,2]-MedianVoff[:,1]],marker="o",ls="--",color="#702963",ms=10)
plt.axhline(y=0,ls="--",color="gray")
plt.ylabel(r"Offset in km/s, (z$_{spec}$-z$_{fresco}$)")
plt.xlabel(r"z$_{spec}$")
plt.ylim(-200,200)
plt.legend()
plt.show()


In [ ]:
np.mean(MedianVoff[:,1])

In [ ]:
sns.set_style('ticks')

num_plots = len(bin_centers)
num_cols = 3  # Number of columns in the subplots grid
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows

# Create the subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(32, 22))
if num_rows > 1:
    axs = axs.flatten()

for i,dat in enumerate(binned_voff):
    dat=np.array(dat)
    dat=dat[(dat<1000)&(dat>-1000)]

    ax=axs[i]
    ax.hist(dat,bins=20,color="#702963",histtype='step',lw=4)
    ax.axvline(x=np.percentile(dat,50),color="black",lw=5,ls="--")
    ax.axvline(x=np.percentile(dat,16),color="gray",lw=3,ls="--")
    ax.axvline(x=np.percentile(dat,84),color="gray",lw=3,ls="--")

    text="Offset : "+str(np.round(np.percentile(dat,50),1))+" km/s"
    ax.text(0.8, 0.8, text, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    ax.set_xticks([-500, -250, 0, 250, 500])
    ax.set_xticklabels(['-500', '-250', '0', '250', '500'])
axs[10].set_xlabel(r"Offset in km/s, (z$_{spec}$-z$_{fresco}$)")
plt.show()



## Running Median

In [ ]:
import pandas as pd

toSort=np.transpose(np.array([Merged_data,Merged_dz,Merged_voff]))
Sorted=np.transpose(toSort[np.lexsort(np.transpose(toSort)[::-1])])

SMerged_data,SMerged_dz,SMerged_voff=Sorted

window=50


Rolled_dz=pd.Series(SMerged_dz).rolling(window,min_periods=0).median().values
Rolled_voff=pd.Series(SMerged_voff).rolling(window,min_periods=0).median().values

R=Rolled_16voff=pd.Series(SMerged_voff).rolling(window,min_periods=0)
Rolled_16voff=pd.Series(SMerged_voff).rolling(window,min_periods=0).quantile(0.16).values
Rolled_84voff=pd.Series(SMerged_voff).rolling(window,min_periods=0).quantile(0.84).values


plt.scatter(SMerged_data,SMerged_voff)
plt.show()

plt.scatter(SMerged_data,SMerged_voff,alpha=0.3)
plt.plot(SMerged_data,Rolled_voff,color="red",lw=3,ls="--")
plt.plot(SMerged_data,Rolled_16voff,color="red",lw=3,ls="--")
plt.plot(SMerged_data,Rolled_84voff,color="red",lw=3,ls="--")
plt.axhline(y=0)
plt.ylabel(r"Voff kms")
plt.xlabel("Wavelength of max line [A]")
plt.show()




In [ ]:
binsize=1000
bin_edges = np.arange(min(SMerged_data), max(SMerged_data)+binsize, binsize)
bin_centers = bin_edges_to_centers(bin_edges)
binned_Data=bin_data(SMerged_data,SMerged_dz,bin_edges)
binned_voff=bin_data(SMerged_data,SMerged_voff,bin_edges)

MedianData=[]
for d in binned_Data:
    MedianData.append( np.percentile(d,q=[16,50,84]) )
MedianData=np.array(MedianData)

MedianVoff=[]
for d in binned_voff:
    MedianVoff.append( np.percentile(d,q=[16,50,84]) )
MedianVoff=np.array(MedianVoff)


In [ ]:
difference=-Tools.kmsToangstrom2(np.array(Rolled_voff),np.array(SMerged_data))
plt.hist(difference,bins=100,range=[-100,100])
plt.xlabel("Difference Wavelength")

In [ ]:
plt.scatter(SMerged_data,SMerged_voff,alpha=0.1)
plt.errorbar(bin_centers,MedianVoff[:,1],yerr=[MedianVoff[:,1]-MedianVoff[:,0],MedianVoff[:,2]-MedianVoff[:,1]],marker="o",ls="--",color="#702963",ms=10)
plt.axhline(y=0,ls="--",color="gray")
plt.ylabel(r"Offset in km/s, (z$_{spec}$-z$_{fresco}$)")
plt.xlabel(r"z$_{spec}$")
plt.ylim(-200,200)
plt.legend()
plt.show()

In [ ]:
sns.set_theme(style="white",font_scale = 1.5)
plt.rcParams["figure.figsize"] = (15,12)

plt.scatter(SMerged_data,SMerged_voff,alpha=0.3)
#plt.errorbar(bin_centers,MedianVoff[:,1],yerr=[MedianVoff[:,1]-MedianVoff[:,0],MedianVoff[:,2]-MedianVoff[:,1]],marker="o",ls="--",color="#702963",ms=5,zorder=10,label="Binned data")

plt.plot(SMerged_data,Rolled_voff,color="red",lw=3,ls="-",label="Rolled median")
plt.fill_between(SMerged_data,Rolled_16voff, Rolled_84voff,color="red",alpha=0.1)
plt.plot(SMerged_data,Rolled_16voff,ls="-",color="red",lw=0.5)
plt.plot(SMerged_data,Rolled_84voff,ls="-",color="red",lw=0.5)


plt.axhline(y=np.median(Rolled_voff),zorder=0,color="black",ls="--",label="Median Offset "+str(np.round(np.median(Rolled_voff),1))+"km/s")
#plt.axhline(y=0,color="gray",lw=3,ls="--",zorder=0)
plt.grid()
plt.legend()
plt.ylabel(r"Offset in km/s, (z$_{spec}$-z$_{fresco}$)")
plt.xlabel("Wavelength of max line [A]")
plt.ylim(-500,500)
plt.show()

In [ ]:
sns.set_style('ticks')

num_plots = len(bin_centers)
num_cols = 3  # Number of columns in the subplots grid
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows

# Create the subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(32, 22))
if num_rows > 1:
    axs = axs.flatten()

for i,dat in enumerate(binned_voff):
    dat=np.array(dat)
    dat=dat[(dat<1000)&(dat>-1000)]

    ax=axs[i]
    ax.hist(dat,bins=20,color="#702963",histtype='step',lw=4)
    ax.axvline(x=np.percentile(dat,50),color="black",lw=5,ls="--")
    ax.axvline(x=np.percentile(dat,16),color="gray",lw=3,ls="--")
    ax.axvline(x=np.percentile(dat,84),color="gray",lw=3,ls="--")

    text="Offset : "+str(np.round(np.percentile(dat,50),1))+" km/s"
    ax.text(0.8, 0.8, text, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    ax.set_xticks([-500, -250, 0, 250, 500])
    ax.set_xticklabels(['-500', '-250', '0', '250', '500'])
axs[10].set_xlabel(r"Offset in km/s, (z$_{spec}$-z$_{fresco}$)")
plt.show()

## Modules

In [ ]:
#Data loaded from ExpositionsGrizli.ipynb

modules=np.load("../Catalogs/ExpositionsGrizli_modNums.npy")
As,Bs,mix,var=[],[],[],[]
for m,of in zip(modules,Merged_voff):
    if m<-0.9:
        As.append(of)
        plt.scatter(m,of,color="green")
    elif m>0.9:
        Bs.append(of)
        plt.scatter(m,of,color="red")
    elif m<0.01 and m>-0.01:
        mix.append(of)
        plt.scatter(m,of,color="purple")
    else:
        var.append(of)
        plt.scatter(m,of,color="grey")


plt.scatter(-5000,5000,s=200,color="green",label="A sided")
plt.scatter(-5000,5000,s=200,color="red",label="B sided")
plt.scatter(-5000,5000,s=200,color="purple",label="Equally Mixed")
plt.scatter(-5000,5000,s=200,color="grey",label="Unequally mixed")


plt.axhline(y=np.median(Bs),ls="--",color="red")
plt.axhline(y=np.median(mix),ls="--",color="purple")
plt.axhline(y=np.median(As),ls="--",color="green")
plt.axhline(y=0,ls="-",color="black")
plt.title("A = -1 , B = 1")
plt.xlabel("Module")
plt.ylabel("Systematic Offset [km/s]")
plt.legend()
plt.xlim(-1.1,1.1)
plt.ylim(-500,500)
plt.show()

In [ ]:
modules

In [ ]:
toSort=np.transpose(np.array([Merged_voff,modules]))
Sorted=np.transpose(toSort[np.lexsort(np.transpose(toSort)[:])])

mask= (Sorted[1]>-0.9) & (Sorted[1]<0.9) & (Sorted[1]!=0)
SMerged_voff,SMerged_modules=Sorted[0][mask],Sorted[1][mask]

window=30

Rolled_modules=pd.Series(SMerged_modules).rolling(window,min_periods=0).median().values
Rolled_voff=pd.Series(SMerged_voff).rolling(window,min_periods=0).median().values

def calculate_slope(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return (y2 - y1) / (x2 - x1)

def calculate_intercept(point, slope):
    x, y = point
    return y - slope * x

def plot_line(point1, point2):
    slope = calculate_slope(point1, point2)
    intercept = calculate_intercept(point1, slope)
    x_values = [point1[0], point2[0]]
    y_values = [point1[1], point2[1]]
    plt.plot(x_values, y_values, label='Straight Line between points',zorder=0,color="black")
    plt.xlabel('x')
    plt.ylabel('y')



for m,of in zip(modules,Merged_voff):
    if m<-0.9:
        plt.scatter(m,of,color="green")
    elif m>0.9:
        plt.scatter(m,of,color="red")
    elif m<0.01 and m>-0.01:
        plt.scatter(m,of,color="purple")
    else:
        plt.scatter(m,of,color="grey")


plt.scatter(-5000,5000,s=200,color="green",label="A sided")
plt.scatter(-5000,5000,s=200,color="red",label="B sided")
plt.scatter(-5000,5000,s=200,color="purple",label="Equally Mixed")
plt.scatter(-5000,5000,s=200,color="grey",label="Unequally mixed")

plt.plot(SMerged_modules,Rolled_voff,color="black",lw=3,ls="--",label="Rolled median")
plt.scatter(-1,-6,s=150)
plt.scatter(1,113,s=150)
plt.scatter(0,31,s=150)
plot_line((-1,-6),(1,113))
#plt.axhline(y=np.median(Bs),ls="--",color="red")
#plt.axhline(y=np.median(mix),ls="--",color="purple")
#plt.axhline(y=np.median(As),ls="--",color="green")
#plt.axhline(y=0,ls="-",color="black")
plt.title("A = -1 , B = 1")
plt.xlabel("Module")
plt.ylabel("Systematic Offset [km/s]")
plt.legend(fontsize=12)
plt.xlim(-1.1,1.1)
plt.ylim(-500,500)
plt.show()

In [ ]:

plt.hist(As,alpha=0.5,range=[-500,500],bins=20,color="green")
#plt.hist(Bs,alpha=0.5,range=[-500,500],bins=20,color="red")
#plt.hist(mix,alpha=0.5,range=[-500,500],bins=20,color="purple")
#plt.hist(var,alpha=0.5,range=[-500,500],bins=20,color="gray")

In [ ]:
AB=[]

for m,of,w in zip(modules,Merged_voff,Merged_data):
    if m<-0.9:
        AB.append({"Module":"A","voff":of,"wave":w})
    elif m>0.9:
        AB.append({"Module":"B","voff":of,"wave":w})


In [ ]:
Module="B"
window=30

Mvoff=[gal["voff"] for gal in AB if gal["Module"]==Module]
wave=[gal["wave"] for gal in AB if gal["Module"]==Module]

toSort=np.transpose(np.array([wave,Mvoff]))
Sorted=np.transpose(toSort[np.lexsort(np.transpose(toSort)[::-1])])

wave,Mvoff=Sorted


R_voff=pd.Series(Mvoff).rolling(window,min_periods=0).median().values
R_16voff=pd.Series(Mvoff).rolling(window,min_periods=0).quantile(0.16).values
R_84voff=pd.Series(Mvoff).rolling(window,min_periods=0).quantile(0.84).values

sns.set_theme(style="white",font_scale = 1.8)
plt.rcParams["figure.figsize"] = (15,8)

plt.scatter(wave,Mvoff,alpha=0.3)


plt.plot(wave,R_voff,color="red",lw=3,ls="-",label="Rolled median")
plt.fill_between(wave,R_16voff, R_84voff,color="red",alpha=0.1)
plt.plot(wave,R_16voff,ls="-",color="red",lw=0.5)
plt.plot(wave,R_84voff,ls="-",color="red",lw=0.5)


plt.axhline(y=np.median(Mvoff),zorder=0,color="black",ls="--",label="Median Offset "+str(np.round(np.median(Mvoff),1))+"km/s")
plt.axhline(y=0,color="black")
#plt.axhline(y=0,color="gray",lw=3,ls="--",zorder=0)
plt.grid()
plt.legend()
plt.ylabel(r"Offset in km/s, (z$_{spec}$-z$_{fresco}$)")
plt.xlabel("Wavelength of max line [A]")
plt.title("Searching for wavelength dependency, Module : "+Module)
#plt.ylim(-500,500)
plt.show()





## Map

In [ ]:
import matplotlib.colors as colors

cmap = plt.get_cmap('seismic')
new_cmap = truncate_colormap(cmap, 0.2, 0.8)



ra=np.array(Merged_coord)[:,0]
dec=np.array(Merged_coord)[:,1]
Z=np.array(Merged_voff)

mask=ra>100
ra=ra[mask]
dec=dec[mask]
Z=Z[mask]



cmap_range = [-500, 500]
norm = plt.Normalize(*cmap_range)

plt.scatter(dec, ra, c=Z,norm=norm, cmap=new_cmap)
cbar = plt.colorbar()
cbar.set_label('Offset km/s')
plt.title("GOODS-N")
plt.xlabel("DEC")
plt.xlabel("RA")
plt.show()

ra=np.array(Merged_coord)[:,0]
dec=np.array(Merged_coord)[:,1]
Z=np.array(Merged_voff)

mask=ra<100
ra=ra[mask]
dec=dec[mask]
Z=Z[mask]



cmap_range = [-500, 500]
norm = plt.Normalize(*cmap_range)

plt.scatter(dec, ra, c=Z,norm=norm, cmap=new_cmap)
cbar = plt.colorbar()
cbar.set_label('Offset km/s')
plt.title("GOODS-S")
plt.xlabel("DEC")
plt.xlabel("RA")
plt.show()


In [ ]:
ra

In [ ]:
Merged_coord[1]

In [ ]:
np.shape(Merged_coord)